# Limpieza de datos

In [39]:
#Paquetes
import pandas as pd
import numpy as np

## Datos de incendios

In [40]:
#Carga de datos
df = pd.read_csv('/home/dsc/Galician_fires/fires-all.csv')
#Selección de la comunidad de Galicia
df = df[df['idcomunidad']==3]
df.dtypes

id                   int64
superficie         float64
fecha               object
lat                float64
lng                float64
latlng_explicit      int64
idcomunidad          int64
idprovincia          int64
idmunicipio          int64
municipio           object
causa                int64
causa_supuesta     float64
causa_desc           int64
muertos            float64
heridos            float64
time_ctrl            int64
time_ext             int64
personal             int64
medios               int64
gastos             float64
perdidas           float64
dtype: object

In [41]:
#Campo año
df['fecha']=df['fecha'].astype('datetime64[ns, US/Eastern]')
df['ano']=df['fecha'].dt.year

In [42]:
df['idprovincia']=df['idprovincia'].astype(str)
df['idmunicipio']=df['idmunicipio'].astype(str)

df=df[~df['lat'].isna()]

In [43]:
#Eliminamos coordenadas incorrectas
df=df[df['id']!=2005270158]
df=df[df['id']!=2004271037]

#### Generación del código postal

In [44]:
df['len']=df.idmunicipio.str.len()
y=[]
for x in (3-df['len']):
    y.append('0'*x)
df['ceros']=y

df['codigo']=df.idprovincia.astype(str)+df.ceros.astype(str)+df.idmunicipio.astype(str)
df['codigo']= pd.to_numeric(df['codigo'], downcast='integer')

## Datos de superficies por municipios

In [45]:
#Ourense
ou = pd.read_excel('/home/dsc/Galician_fires/Complementary_data/Distribucion_terras/Distribucion_terras_ourense.xlsx')
ou.columns = map(lambda x: str(x).lower(), ou.columns)
ou['municipio'] = ou['municipio'].str.upper()
#Pontevedra
po= pd.read_excel('/home/dsc/Galician_fires/Complementary_data/Distribucion_terras/Distribucion_terras_pontevedra.xlsx')
po.columns = map(lambda x: str(x).lower(), po.columns)
po['municipio'] = po['municipio'].str.upper()
#A Coruña
co= pd.read_excel('/home/dsc/Galician_fires/Complementary_data/Distribucion_terras/Distribucion_terras_coruña.xlsx')
co.columns = map(lambda x: str(x).lower(), co.columns)
co['municipio'] = co['municipio'].str.upper()
#Lugo
lu= pd.read_excel('/home/dsc/Galician_fires/Complementary_data/Distribucion_terras/Distribucion_terras_lugo.xlsx')
lu.columns = map(lambda x: str(x).lower(), lu.columns)
lu['municipio'] = lu['municipio'].str.upper()

gal = [co, lu, ou, po]
gal = pd.concat(gal)

#Selección de la superficie por municipios
gal['codigo_str']=gal['codigo'].astype(str)
gal['length'] = gal.codigo_str.str.len()
gal=gal[gal['length']==5]

#Cambio de nombre de columnas
gal = gal.rename(columns={'superficie total do concello' : 'superficie_total_concello',
                         'superficie forestal total':'superficie_forestal_total',
                         'forestal arborizado':'forestal_arborizado',
                         'pasteiro, pasteiro arbustivo e mato':'pasteiro_pasteiro_arbustivo_mato',
                         'superficie de cultivo e prado':'superficie_cultivo_prado',
                         'outras superficies':'outras_superficies'})

gal

,codigo,municipio,ano,superficie_total_concello,superficie_forestal_total,forestal_arborizado,pasteiro_pasteiro_arbustivo_mato,superficie_cultivo_prado,outras_superficies,codigo_str,length
28,15006,ARZÚA,2006,15543.0,6969.0,6618.0,351.0,6958.0,1616.0,15006,5
29,15006,ARZÚA,2007,15543.0,7069.0,6653.0,417.0,6890.0,1583.0,15006,5
30,15006,ARZÚA,2008,15543.0,6911.0,6590.0,321.0,7041.0,1590.0,15006,5
31,15006,ARZÚA,2009,15543.0,6938.0,6634.0,303.0,7013.0,1592.0,15006,5
32,15006,ARZÚA,2010,15543.0,6936.0,6621.0,315.0,6973.0,1633.0,15006,5
...,...,...,...,...,...,...,...,...,...,...,...
1031,36057,VIGO,2015,10906.0,3731.0,NaN,NaN,1316.0,5859.0,36057,5
1032,36057,VIGO,2016,10906.0,3727.0,NaN,NaN,1315.0,5864.0,36057,5
1033,36057,VIGO,2017,10906.0,3728.0,NaN,NaN,1314.0,5864.0,36057,5
1034,36057,VIGO,2018,10906.0,3728.0,NaN,NaN,1312.0,5865.0,36057,5


### Unión de incendios con superficies

In [46]:
df.dtypes, gal.dtypes

(id                                      int64
 superficie                            float64
 fecha              datetime64[ns, US/Eastern]
 lat                                   float64
 lng                                   float64
 latlng_explicit                         int64
 idcomunidad                             int64
 idprovincia                            object
 idmunicipio                            object
 municipio                              object
 causa                                   int64
 causa_supuesta                        float64
 causa_desc                              int64
 muertos                               float64
 heridos                               float64
 time_ctrl                               int64
 time_ext                                int64
 personal                                int64
 medios                                  int64
 gastos                                float64
 perdidas                              float64
 ano         

In [48]:
df[['lat','lng']].drop_duplicates()

,lat,lng
1202,43.703581,-8.038777
1203,42.936918,-9.114350
1204,42.643031,-8.939252
1205,43.186836,-8.685470
1206,42.917476,-9.082862
...,...,...
81478,42.103959,-8.522512
81479,42.157695,-8.397080
81480,42.554722,-8.018586
81481,42.243441,-8.322375


In [30]:
df = pd.merge(df, gal, on = ['ano','codigo'], how ='left')

## Selección de columnas a analizar

In [31]:
df['causa_desc'] = df['causa'].apply(lambda x: 'Rayo' if x==1
                                             else ('Descuidos humanos' if x==2 
                                                   else ('Accidentes' if x==3
                                                         else ('Intencionado' if x==4
                                                              else ('Desconodico' if x==5
                                                                   else 'Reproducido')))))
df['provincia'] = df['idprovincia'].apply(lambda x: 'A CORUÑA' if x==str(15) else ('OURENSE' if x==str(32) else ('LUGO' if x==str(27) else 'PONTEVEDRA')))

df=df[['id', 'superficie', 'fecha', 'lat', 'lng', 
       'idprovincia','provincia', 'idmunicipio', 'municipio_x','causa','causa_desc', 'time_ctrl',
       'time_ext', 'personal', 'medios', 'gastos', 'perdidas', 'codigo',
       'superficie_total_concello', 'superficie_forestal_total']]

df = df.rename(columns={'municipio_x' : 'municipio'})
df.describe()

,id,superficie,lat,lng,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas,codigo,superficie_total_concello,superficie_forestal_total
count,2.458300e+04,24583.000000,24583.000000,24583.000000,24583.000000,24583.000000,24583.000000,24583.000000,24583.000000,4424.000000,1.644900e+04,24583.000000,10415.000000,10415.000000
mean,2.006205e+09,14.925911,42.530579,-7.931967,4.004027,227.761665,379.794696,18.857300,3.262905,8146.943264,3.143277e+04,28011.766424,13110.506385,8822.887086
std,3.951160e+06,95.154664,0.438370,0.586974,0.614903,364.897337,604.735691,24.215916,4.688116,22982.279745,3.196188e+05,7577.560411,7269.637373,5322.887466
min,2.001150e+09,1.000000,41.831486,-9.293617,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000e+00,15001.000000,231.000000,126.000000
25%,2.003151e+09,1.500000,42.181715,-8.372554,4.000000,88.000000,140.000000,7.000000,1.000000,1049.750000,0.000000e+00,27009.000000,8318.000000,5190.000000
50%,2.005321e+09,2.500000,42.429862,-7.942447,4.000000,135.000000,225.000000,13.000000,2.000000,2263.000000,1.611000e+03,32029.000000,11459.000000,7625.000000
75%,2.009321e+09,6.000000,42.863732,-7.430299,4.000000,230.000000,380.000000,22.000000,4.000000,6789.500000,6.446000e+03,32076.000000,17142.000000,10431.000000
max,2.015361e+09,7316.770000,43.761264,-6.607656,6.000000,10125.000000,12637.000000,855.000000,205.000000,564585.000000,1.542632e+07,36901.000000,43846.000000,34414.000000


In [32]:
def Q1(x):
    return x.quantile(0.25)
def Q2(x):
    return x.quantile(0.5)
def Q3(x):
    return x.quantile(0.75)
groupby = df.groupby(['provincia']).agg({'superficie': ['sum','mean','min',Q1,Q2,Q3,'max','std']})

groupby

superficie                                                      
                  sum       mean  min   Q1   Q2     Q3      max         std
provincia                                                                  
A CORUÑA     78439.18  13.622643  1.0  1.5  2.5  5.440  2842.00   79.447218
LUGO         40310.12  11.748796  1.0  1.5  2.5  6.255  2364.67   59.551630
OURENSE     165740.40  14.501741  1.0  1.5  2.5  6.000  3236.70   79.256080
PONTEVEDRA   82433.96  20.790406  1.0  1.6  3.0  7.500  7316.77  160.493902

In [33]:
df

,id,superficie,fecha,lat,lng,idprovincia,provincia,idmunicipio,municipio,causa,causa_desc,time_ctrl,time_ext,personal,medios,gastos,perdidas,codigo,superficie_total_concello,superficie_forestal_total
0,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,Descuidos humanos,235,270,14,2,NaN,7013.0,15022,NaN,NaN
1,2001150088,1.50,2001-02-24 00:00:00-05:00,42.936918,-9.114350,15,A CORUÑA,34,DUMBRÍA,4,Intencionado,470,530,14,1,NaN,1497.0,15034,NaN,NaN
2,2001150090,3.00,2001-02-25 00:00:00-05:00,42.643031,-8.939252,15,A CORUÑA,67,"POBRA DO CARAMIÑAL, A",4,Intencionado,185,220,14,3,NaN,1882.0,15067,NaN,NaN
3,2001150094,1.50,2001-02-25 00:00:00-05:00,43.186836,-8.685470,15,A CORUÑA,19,CARBALLO,4,Intencionado,125,135,5,0,NaN,1028.0,15019,NaN,NaN
4,2001150111,3.80,2001-02-25 00:00:00-05:00,42.917476,-9.082862,15,A CORUÑA,45,MAZARICOS,4,Intencionado,1050,1051,14,1,NaN,3119.0,15045,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24578,2015360747,1.45,2015-09-06 00:00:00-04:00,42.103959,-8.522512,36,PONTEVEDRA,50,SALVATERRA DE MIÑO,4,Intencionado,92,486,16,3,2012.0,6198.0,36050,6254.0,3481.0
24579,2015360751,1.00,2015-09-06 00:00:00-04:00,42.157695,-8.397080,36,PONTEVEDRA,34,"NEVES, AS",4,Intencionado,404,420,15,1,1509.0,430.0,36034,6550.0,4580.0
24580,2015360770,5.94,2015-09-08 00:00:00-04:00,42.554722,-8.018586,36,PONTEVEDRA,16,DOZÓN,4,Intencionado,87,194,18,3,1541.0,0.0,36016,7423.0,4702.0
24581,2015360794,3.20,2015-09-28 00:00:00-04:00,42.243441,-8.322375,36,PONTEVEDRA,13,COVELO,4,Intencionado,89,428,21,4,5805.0,0.0,36013,13190.0,10256.0


In [34]:
df.to_csv('data.csv')

In [38]:
df[['lat','lng']].drop_duplicates()

,lat,lng
0,43.703581,-8.038777
1,42.936918,-9.114350
2,42.643031,-8.939252
3,43.186836,-8.685470
4,42.917476,-9.082862
...,...,...
24578,42.103959,-8.522512
24579,42.157695,-8.397080
24580,42.554722,-8.018586
24581,42.243441,-8.322375
